# Train classifier to estimate analysis efficiency vs gen variable

- Actual training code in train.py
- Classifier based on sklearn. Default is GradientBoostedClassifier, 
    but can be specified at run time.
- Use XGBoost for recoPt and recoNjets2p5 training (multi-threading)

## Load libraries

In [1]:
import train as tn
#reload(tn)

import plotting
reload(plotting)

import matplotlib.pyplot as plt
plt.style.use('seaborn-poster')
%matplotlib inline

import numpy as np

from pprint import pprint

import os
import json
import importlib

import util as ut
reload(ut)

import angles as ang

Welcome to ROOTaaS 6.06/05


## Instantiate helper class

Data are read from ROOT trees and converted into pandas data frames.  
The loading function makes sure that all the needed columns have been read from the trees, otherwise it rebilds the data frame.


#### In the following cell the parameters are set up. The training data will be read from a root tree located in dataDir. The branches of gen and reco events are the default ones (look at the help function for detailed information). dataFiles gives an extra proces label to the different Higgs production mechanisms.

In [2]:
ut.defaultParameters(dataDir="/mnt/t3nfs01/data01/shome/jandrejk/higgs_model_dep/MoriondAnalysis/data", 
                     classifiers=['class','recoNjets2p5'],#,'recoPt','recoNjets2p5'], #['class',
                          load = True,
                     inputName = "1clfs_pt20cut",
                       outName = "3clfs",
                        outDir = './classifiers',
                    inputDir = '/mnt/t3nfs01/data01/shome/jandrejk/higgs_model_dep/MoriondAnalysis/classifiers',
                    defineBins = { 'recoPt' : dict(boundaries=[0.,15.,30.,45.,85.,125.,200.,350.,10000.],overflow=False), # do not add overflow automatically
                                  'recoNjets2p5' : dict(boundaries=[-0.5,0.5,1.5,2.5,3.5,100.],overflow=False)
                    #              #'genPt' : dict(boundaries=[0.,15.,30.,45.,85.,125.,200.,350.,10000.],overflow=False), # do not add overflow automatically
                    #              #'genNjets2p5' : dict(boundaries=[-0.5,0.5,1.5,2.5,3.5,100.],overflow=False)
                                 },
                      dataFiles=[(0,'output_GluGluHToGG_M125_IA_20GeVgenJetPtcut.root'),                   
                                (1,'output_ttHToGG_M125_IA_20GeVgenJetPtcut.root'),                     
                                (2,'output_VBFHToGG_M125_IA_20GeVgenJetPtcut.root'),                            
                                (3,'output_VHToGG_M125_IA_20GeVgenJetPtcut.root'),
                               ]
                    )



#### We want to use machine learning techniques for classification. For that we need to specify the classifier we want to use like here the GradientBoostingClassifier from sklearn.ensamble. Furthermore the parameters of the used classifier have to be set, e.g. the number of training events, max. tree depth, learning rate and so forth.

In [3]:
ut.params["class"] = [ "sklearn.ensemble.GradientBoostingClassifier", 
                      dict(trainevts= -1,
                           max_depth=5,learning_rate=0.2,n_estimators=200,
                        min_weight_fraction_leaf=1e-3)
]

"""
ut.params['recoPt'] = ["xgboost.sklearn.XGBClassifier",
                       { "Xbr" : ["genPt","absGenRapidity",
                                  'DeltaRLeadGamma0','DeltaRLeadGamma1','DeltaRLeadGamma2',
                                    'DeltaRLeadGamma3','DeltaRLeadGamma4','DeltaRLeadGamma5',
                                    'DeltaRSubleadGamma0','DeltaRSubleadGamma1','DeltaRSubleadGamma2',
                                    'DeltaRSubleadGamma3','DeltaRSubleadGamma4','DeltaRSubleadGamma5',
                                    'DeltaRLeadGammaSubleadGamma',
                                    'DeltaR01','DeltaR02','DeltaR03','DeltaR04','DeltaR05',
                                    'DeltaR12','DeltaR13','DeltaR14','DeltaR15','DeltaR23',
                                    'DeltaR24','DeltaR25','DeltaR34','DeltaR35','DeltaR45'
                                  ],
                        "trainevts" : -1, 
                        "max_depth" : 5,"learning_rate" : 0.1,
                        "n_estimators" : 500,"min_child_weight" : 1e-5,
                        "nthread" : 4}]

"""
""",
                        "cvoptimize" : True,"cv_params_grid" : { "max_depth" : [5, 7, 10],
                                                                 "learning_rate" : [0.05, 0.1, 0.2], 
                                                                 "n_estimators" : [250,500,700,1000],
                                                                 "min_child_weight" : [1e-4, 5e-4, 1e-3],
                                                                 "subsample" : [0.1, 0.2, 0.5, 1.]}, 
                        "cv_nfolds" : 5, "cv_niter" : 5,      
"""


ut.params['recoNjets2p5'] =  ["xgboost.sklearn.XGBClassifier",
                        { "Xbr" : ["genJet2p5Pt0",  "absGenJet2p5Rapidity0",
                                   "genJet2p5Pt1", "absGenJet2p5Rapidity1",
                                   "genJet2p5Pt2", "absGenJet2p5Rapidity2",
                                   "genJet2p5Pt3", "absGenJet2p5Rapidity3",
                                   "genJet2p5Pt4",  "absGenJet2p5Rapidity4",
                                   "genJet2p5Pt5",  "absGenJet2p5Rapidity5",
                                   "genJet2p5pt20Pt0",  "absGenJet2p5pt20Rapidity0",
                                   "genJet2p5pt20Pt1", "absGenJet2p5pt20Rapidity1",
                                   "genJet2p5pt20Pt2", "absGenJet2p5pt20Rapidity2",
                                   "genJet2p5pt20Pt3", "absGenJet2p5pt20Rapidity3",
                                   "genJet2p5pt20Pt4",  "absGenJet2p5pt20Rapidity4",
                                   "genJet2p5pt20Pt5",  "absGenJet2p5pt20Rapidity5",
                                   "genPt","absGenRapidity",
                                   "genNjets2p5",
                                   'genLeadGenIso','genSubleadGenIso',
                                 'genAbsCosDeltaAlpha2p5pt2001','genAbsCosDeltaAlpha2p5pt2002',
                                 'genAbsCosDeltaAlpha2p5pt2003','genAbsCosDeltaAlpha2p5pt2004',
                                 'genAbsCosDeltaAlpha2p5pt2005','genAbsCosDeltaAlpha2p5pt2012',
                                 'genAbsCosDeltaAlpha2p5pt2013','genAbsCosDeltaAlpha2p5pt2014',
                                 'genAbsCosDeltaAlpha2p5pt2015','genAbsCosDeltaAlpha2p5pt2023',
                                 'genAbsCosDeltaAlpha2p5pt2024','genAbsCosDeltaAlpha2p5pt2025',
                                 'genAbsCosDeltaAlpha2p5pt2034','genAbsCosDeltaAlpha2p5pt2035',
                                 'genAbsCosDeltaAlpha2p5pt2045',
                                 'genAbsCosDeltaPhi2p5pt2001','genAbsCosDeltaPhi2p5pt2002',
                                 'genAbsCosDeltaPhi2p5pt2003','genAbsCosDeltaPhi2p5pt2004',
                                 'genAbsCosDeltaPhi2p5pt2005','genAbsCosDeltaPhi2p5pt2012',
                                 'genAbsCosDeltaPhi2p5pt2013','genAbsCosDeltaPhi2p5pt2014',
                                 'genAbsCosDeltaPhi2p5pt2015','genAbsCosDeltaPhi2p5pt2023',
                                 'genAbsCosDeltaPhi2p5pt2024','genAbsCosDeltaPhi2p5pt2025',
                                 'genAbsCosDeltaPhi2p5pt2034','genAbsCosDeltaPhi2p5pt2035',
                                 'genAbsCosDeltaPhi2p5pt2045'
                                  ],                         
            "trainevts" : -1, "max_depth" : 7,
            "learning_rate" : 0.1,"n_estimators" : 500,
            "min_child_weight" : 1e-5,
            "nthread" : 8 }]

## The specified genBranches are the input features needed for the training (including the weights). When the classifier is loaded later one has to make sure that the genBranches match the one specified here. 

In [5]:
#set the parameters "class" defined above
ut.setParams()
# manual fix in order that the json file doesn't overwrite inputDir
ut.params['inputDir'] = "/mnt/t3nfs01/data01/shome/jandrejk/higgs_model_dep/MoriondAnalysis/classifiers"


ut.params['genpfx'] = 'genDiphotonDumper/trees/InsideAcceptance_125_13TeV'
ut.params['pfx'] = 'tagsDumper/trees/InsideAcceptance_125_13TeV'

genBr = ang.AppendGenJetVariableNames(['genPt','genRapidity','weight','puweight',
                            'genNjets2p5','genLeadGenIso','genSubleadGenIso'])  
genBr = ang.AppendPhotonVariableNames(genBr)
ut.params['genBranches'] = genBr


ut.params['genBranches'] = ['genPt', 'genRapidity', 
                            'genJet2p5Pt0', 'genJet2p5Pt1', 'genJet2p5Pt2', 'genJet2p5Pt3', 
                            'genJet2p5Pt4', 'genJet2p5Pt5', 'genJet2p5Rapidity0', 'genJet2p5Rapidity1', 
                            'genJet2p5Rapidity2', 'genJet2p5Rapidity3', 'genJet2p5Rapidity4', 
                            'genJet2p5Rapidity5', 'weight', 'genNjets2p5', 'genLeadGenIso', 'genSubleadGenIso', 
                            "genJet2p5pt20Pt0",  "genJet2p5pt20Rapidity0",
                                   "genJet2p5pt20Pt1", "genJet2p5pt20Rapidity1",
                                   "genJet2p5pt20Pt2", "genJet2p5pt20Rapidity2",
                                   "genJet2p5pt20Pt3", "genJet2p5pt20Rapidity3",
                                   "genJet2p5pt20Pt4",  "genJet2p5pt20Rapidity4",
                                   "genJet2p5pt20Pt5",  "genJet2p5pt20Rapidity5",
                                 'genAbsCosDeltaAlpha2p5pt2001','genAbsCosDeltaAlpha2p5pt2002',
                                 'genAbsCosDeltaAlpha2p5pt2003','genAbsCosDeltaAlpha2p5pt2004',
                                 'genAbsCosDeltaAlpha2p5pt2005','genAbsCosDeltaAlpha2p5pt2012',
                                 'genAbsCosDeltaAlpha2p5pt2013','genAbsCosDeltaAlpha2p5pt2014',
                                 'genAbsCosDeltaAlpha2p5pt2015','genAbsCosDeltaAlpha2p5pt2023',
                                 'genAbsCosDeltaAlpha2p5pt2024','genAbsCosDeltaAlpha2p5pt2025',
                                 'genAbsCosDeltaAlpha2p5pt2034','genAbsCosDeltaAlpha2p5pt2035',
                                 'genAbsCosDeltaAlpha2p5pt2045',
                                 'genAbsCosDeltaPhi2p5pt2001','genAbsCosDeltaPhi2p5pt2002',
                                 'genAbsCosDeltaPhi2p5pt2003','genAbsCosDeltaPhi2p5pt2004',
                                 'genAbsCosDeltaPhi2p5pt2005','genAbsCosDeltaPhi2p5pt2012',
                                 'genAbsCosDeltaPhi2p5pt2013','genAbsCosDeltaPhi2p5pt2014',
                                 'genAbsCosDeltaPhi2p5pt2015','genAbsCosDeltaPhi2p5pt2023',
                                 'genAbsCosDeltaPhi2p5pt2024','genAbsCosDeltaPhi2p5pt2025',
                                 'genAbsCosDeltaPhi2p5pt2034','genAbsCosDeltaPhi2p5pt2035',
                                 'genAbsCosDeltaPhi2p5pt2045'
                            ]


ut.params['recoBranches'] = ['recoPt','recoRapidity','recoNjets2p5']



#ut.params['recoBranches'] = ['recoPt','recoRapidity']

entered config files named my_train_config
hi
None


In [7]:
reload(tn)
%time effFitter = ut.loadOrMake()

Load object with the name 1clfs_pt20cut and the following paramters 
{'class': ['sklearn.ensemble.GradientBoostingClassifier',
           {'learning_rate': 0.2,
            'max_depth': 5,
            'min_weight_fraction_leaf': 0.001,
            'n_estimators': 200,
            'trainevts': -1}],
 'classifiers': ['class', 'recoNjets2p5'],
 'clean': [],
 'dataDir': '/mnt/t3nfs01/data01/shome/jandrejk/higgs_model_dep/MoriondAnalysis/data',
 'dataFiles': [(0, 'output_GluGluHToGG_M125_IA_20GeVgenJetPtcut.root'),
               (1, 'output_ttHToGG_M125_IA_20GeVgenJetPtcut.root'),
               (2, 'output_VBFHToGG_M125_IA_20GeVgenJetPtcut.root'),
               (3, 'output_VHToGG_M125_IA_20GeVgenJetPtcut.root')],
 'dataFname': 'output_InsideAcceptance_125.root',
 'defineBins': {'recoNjets2p5': {'boundaries': [-0.5,
                                                0.5,
                                                1.5,
                                                2.5,
                

# Do you want to re-weight

In [1]:
reweight = False

In [9]:
def reweightUniform (weight,Cat,factor) :
    return weight / factor[int(Cat)]

In [12]:
effFitter.df['starting_weight'] = df['weight']

In [13]:
if reweight :
    FACTOR = df.groupby('recoNjets2p5Cat')['absweight'].sum()
    print FACTOR

    effFitter.df['weight'] = df[['absweight','recoNjets2p5Cat']].apply(lambda x : reweightUniform(*x,factor=FACTOR),axis=1)

    effFitter.df['absweight'] = abs(df['weight'].values)


    plt.hist((effFitter.df['recoNjets2p5Cat']),weights=effFitter.df['absweight'],bins=np.linspace(-1,15,17)-.5)
    plt.xlabel(r'reco-$N_{\mathrm{jets}}$')
    plt.show()

recoNjets2p5Cat
-1.0     35.640457
 0.0      8.121690
 1.0     13.152004
 2.0     27.284676
 3.0      6.068151
 4.0      8.755610
 5.0     17.138542
 6.0      2.789996
 7.0      3.555040
 8.0      6.541688
 9.0      0.961929
 10.0     1.110005
 11.0     1.937537
 12.0     0.663639
 13.0     0.707208
 14.0     1.089471
Name: absweight, dtype: float32


# Adding angular/distance variables to the data frame

In [19]:
#%time effFitter.df = ang.AddAngularVariablesToDataFrame(df=effFitter.df,JetAngles = True, JetGammaAngles=True, GammaGammaAngles=True)
#%time effFitter.df = ang.AddDistanceToDataFrame(df=effFitter.df)
#%time effFitter.df = ang.AddAntiKTDistance(df=effFitter.df)

## Train classifiers

###  make sure that the trained classifers have been evaluated

In [22]:
ut.runEvaluation(effFitter)

['class']
class
class
class_prob_0
Index([u'absweight', u'class', u'genAbsCosDeltaAlpha2p5pt2001',
       u'genAbsCosDeltaAlpha2p5pt2002', u'genAbsCosDeltaAlpha2p5pt2003',
       u'genAbsCosDeltaAlpha2p5pt2004', u'genAbsCosDeltaAlpha2p5pt2005',
       u'genAbsCosDeltaAlpha2p5pt2012', u'genAbsCosDeltaAlpha2p5pt2013',
       u'genAbsCosDeltaAlpha2p5pt2014', u'genAbsCosDeltaAlpha2p5pt2015',
       u'genAbsCosDeltaAlpha2p5pt2023', u'genAbsCosDeltaAlpha2p5pt2024',
       u'genAbsCosDeltaAlpha2p5pt2025', u'genAbsCosDeltaAlpha2p5pt2034',
       u'genAbsCosDeltaAlpha2p5pt2035', u'genAbsCosDeltaAlpha2p5pt2045',
       u'genAbsCosDeltaPhi2p5pt2001', u'genAbsCosDeltaPhi2p5pt2002',
       u'genAbsCosDeltaPhi2p5pt2003', u'genAbsCosDeltaPhi2p5pt2004',
       u'genAbsCosDeltaPhi2p5pt2005', u'genAbsCosDeltaPhi2p5pt2012',
       u'genAbsCosDeltaPhi2p5pt2013', u'genAbsCosDeltaPhi2p5pt2014',
       u'genAbsCosDeltaPhi2p5pt2015', u'genAbsCosDeltaPhi2p5pt2023',
       u'genAbsCosDeltaPhi2p5pt2024', u'genAb

### Run the actual training

In [23]:
#try negative weights
%time ut.runTraining(effFitter,useAbsWeight=True)

We need to train the following classifiers recoNjets2p5
Fitting recoNjets2p5
<class 'xgboost.sklearn.XGBClassifier'>
{'nthread': 8, 'learning_rate': 0.1, 'trainevts': 10, 'min_child_weight': 1e-05, 'Xbr': ['genJet2p5Pt0', 'absGenJet2p5Rapidity0', 'genJet2p5Pt1', 'absGenJet2p5Rapidity1', 'genJet2p5Pt2', 'absGenJet2p5Rapidity2', 'genJet2p5Pt3', 'absGenJet2p5Rapidity3', 'genJet2p5Pt4', 'absGenJet2p5Rapidity4', 'genJet2p5Pt5', 'absGenJet2p5Rapidity5', 'genJet2p5pt20Pt0', 'absGenJet2p5pt20Rapidity0', 'genJet2p5pt20Pt1', 'absGenJet2p5pt20Rapidity1', 'genJet2p5pt20Pt2', 'absGenJet2p5pt20Rapidity2', 'genJet2p5pt20Pt3', 'absGenJet2p5pt20Rapidity3', 'genJet2p5pt20Pt4', 'absGenJet2p5pt20Rapidity4', 'genJet2p5pt20Pt5', 'absGenJet2p5pt20Rapidity5', 'genPt', 'absGenRapidity', 'genNjets2p5', 'genLeadGenIso', 'genSubleadGenIso', 'genAbsCosDeltaAlpha2p5pt2001', 'genAbsCosDeltaAlpha2p5pt2002', 'genAbsCosDeltaAlpha2p5pt2003', 'genAbsCosDeltaAlpha2p5pt2004', 'genAbsCosDeltaAlpha2p5pt2005', 'genAbsCosDelta

## Save the output

In [ ]:
#reload(tn)
%time tn.IO.save(effFitter)